In [1]:
from keras.applications import vgg19
import keras
import pandas as pd
import numpy as np
from skimage import io,transform
import os
import sys
model_name = 'VGGNet_RealSpam'

Using TensorFlow backend.


In [2]:
size=128
x = np.load('../Images/SpamReal/spamreal_x%s.npy' % size)
y = np.load('../Images/SpamReal/spamreal_y%s.npy' % size)

In [3]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=7)

In [4]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [5]:
from keras.layers import GlobalAveragePooling2D, Dropout, Dense
from keras.models import Model
base = vgg19.VGG19(include_top=False, weights=None, classes=2, input_shape=(size,size,3), pooling='avg')
temp = base.output
temp = Dropout(0.5)(temp)
predictions = Dense(2, activation= 'softmax')(temp)
model = Model(inputs = base.input, outputs = predictions)

In [6]:
from keras.callbacks import ModelCheckpoint
# rmsp = rmsprop(lr=0.01, decay=0.9, epsilon=1, clipvalue=2)
model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['acc'])

filepath="%s.best.hdf5" % model_name
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

hist = model.fit([x_train], y_train, batch_size=32, epochs=40, validation_data=([x_test], y_test), verbose=1, callbacks=callbacks_list)

Train on 7872 samples, validate on 2625 samples
Epoch 1/40
7872/7872 [==============================] - 36s 5ms/step - loss: 0.6901 - acc: 0.5427 - val_loss: 0.6847 - val_acc: 0.5703

Epoch 00001: val_loss improved from inf to 0.68466, saving model to VGGNet_RealSpam.best.hdf5
Epoch 2/40
7872/7872 [==============================] - 34s 4ms/step - loss: 0.6891 - acc: 0.5427 - val_loss: 0.6838 - val_acc: 0.5703

Epoch 00002: val_loss improved from 0.68466 to 0.68378, saving model to VGGNet_RealSpam.best.hdf5
Epoch 3/40
7872/7872 [==============================] - 34s 4ms/step - loss: 0.6884 - acc: 0.5427 - val_loss: 0.6823 - val_acc: 0.5703

Epoch 00003: val_loss improved from 0.68378 to 0.68234, saving model to VGGNet_RealSpam.best.hdf5
Epoch 4/40
7872/7872 [==============================] - 34s 4ms/step - loss: 0.6865 - acc: 0.5427 - val_loss: 0.6779 - val_acc: 0.5703

Epoch 00004: val_loss improved from 0.68234 to 0.67790, saving model to VGGNet_RealSpam.best.hdf5
Epoch 5/40
7872/7872


Epoch 00038: val_loss improved from 0.27529 to 0.27308, saving model to VGGNet_RealSpam.best.hdf5
Epoch 39/40
7872/7872 [==============================] - 34s 4ms/step - loss: 0.2694 - acc: 0.8929 - val_loss: 0.2568 - val_acc: 0.8960

Epoch 00039: val_loss improved from 0.27308 to 0.25685, saving model to VGGNet_RealSpam.best.hdf5
Epoch 40/40
7872/7872 [==============================] - 35s 4ms/step - loss: 0.2670 - acc: 0.8928 - val_loss: 0.2484 - val_acc: 0.8971

Epoch 00040: val_loss improved from 0.25685 to 0.24836, saving model to VGGNet_RealSpam.best.hdf5


In [7]:
# serialize model to JSON
model_json = model.to_json()
with open("%s.json" % model_name, "w") as json_file:
    json_file.write(model_json)
    
# # serialize weights to HDF5
model.save_weights("%s.h5" % model_name)
print("Saved model to disk")

Saved model to disk


In [6]:
from keras.models import model_from_json
# load json and create model
json_file = open('%s.json' % model_name, 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("%s.h5" % model_name)
print("Loaded model from disk")

model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['acc'])

ResourceExhaustedError: OOM when allocating tensor with shape[3,3,512,512] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: block4_conv2/kernel/Assign = Assign[T=DT_FLOAT, _class=["loc:@block4_conv2/kernel"], use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](block4_conv2/kernel, block4_conv2/random_uniform)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'block4_conv2/kernel/Assign', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py",

In [ ]:
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score, roc_auc_score
import scikitplot as skplt
%matplotlib inline

predicted = model.predict([x_test])
true = []
pred = []
for i in range(len(predicted)):
    p_tmp = predicted[i][1]
    t_tmp = np.argmax(y_test[i], -1)
    pred.append(p_tmp)
    true.append(t_tmp)
pred = np.array(pred)
true = np.array(true)
skplt.metrics.plot_ks_statistic(np.array(true), list(zip(1-pred, pred)))

print("precision_score: ", precision_score(true, pred>=0.488))
print("recall_score: ", recall_score(true, pred>=0.488))
print("f1_score: ", f1_score(true, pred>=0.488))
print("roc_auc_score: ", roc_auc_score(true, pred))

In [9]:
from keras.models import model_from_json
# load json and create model
json_file = open('%s.json' % model_name, 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("%s.best.hdf5" % model_name)
print("Loaded model from disk")

model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['acc'])

Loaded model from disk


In [11]:
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score, roc_auc_score

predicted = model.predict([x_test])
true = []
pred = []
for i in range(len(predicted)):
    p_tmp = np.argmax(predicted[i], axis=-1)
    t_tmp = np.argmax(y_test[i], -1)
    pred.append(p_tmp)
    true.append(t_tmp)

print(precision_score(true, pred))
print(recall_score(true, pred))
print(roc_auc_score(true, pred))
print(f1_score(true, pred))

0.8844086021505376
0.875
0.8944138276553106
0.8796791443850267


In [ ]:
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt
fpr, tpr, _ = roc_curve(true, predicted[:,1])
np.save('fpr_%s.npy' % model_name, fpr)
np.save('tpr_%s.npy' % model_name, tpr)